# 1.

In [1]:
from numpy import array
from numpy import identity
from numpy import zeros
from numpy import shape
from numpy import sqrt
from numpy.linalg import norm

iteration=[5,10,15,20,50,100]
T=array([[1,-1,0,0],[-1,2,-1,0],[0,-1,3,-1],[0,0,-1,4]])

## QR

In [2]:
def Householder(x,y): ## just transform x to y direction, y is not required to be equal to x
    x = x.reshape(-1,1)
    y = y.reshape(-1,1)
    n = x.shape[0]
    e_y = y/norm(y)
    v = x - norm(x)*e_y
    P = identity(n) - 2*v.dot(v.T)/norm(v)/norm(v)
    return P

def QR_decomposition(A):
    n=A.shape[0]
    e=[zeros(n) for i in range(n)]
    ebar=[zeros(n-i) for i in range(n)]
    for i in range(n):
        e[i][0]=1
        ebar[i][0]=1

    a=[A[:,i].reshape(-1,1) for i in range(A.shape[1])]
    abar=[A[i:n,i].reshape(-1,1) for i in range(A.shape[1])]
    P=[identity(n) for i in range(n)]
    Pbar=[identity(n-i) for i in range(n)]
    P_total=identity(n)
    
    for k in range(n):
        abar[k]=A[k:n,k].reshape(-1,1)
        Pbar[k]=Householder(abar[k], ebar[k])
        ## now print P_k
        for i in range(k,n):
            for j in range(k,n):
                P[k][i,j]=Pbar[k][i-k,j-k]
      
        A=P[k].dot(A)
        P_total=P[k].dot(P_total)

    Q=P_total.T
    R=A
    return Q,R

def QR_algorithm(A, iteration_times=20):
    T = A
    for i in range(iteration_times):
        Q,R = QR_decomposition(T)
        T = R.dot(Q)
    return T

for i in iteration:
    print("After %d iterations:"%(i))
    print(QR_algorithm(T,i))
    print()

After 5 iterations:
[[ 4.29276628e+00 -7.21313977e-01  1.67267719e-16 -2.75369668e-16]
 [-7.21313977e-01  3.55611356e+00 -3.34967464e-01 -5.32912976e-15]
 [ 2.72816609e-16 -3.34967464e-01  1.89640130e+00 -3.99652715e-04]
 [ 4.28048001e-19 -4.81990523e-15 -3.99652715e-04  2.54718859e-01]]

After 10 iterations:
[[ 4.73418406e+00 -1.31448547e-01 -1.34674709e-16 -1.57296804e-17]
 [-1.31448547e-01  3.18812610e+00 -1.85822775e-02  1.60528478e-11]
 [ 9.07306973e-17 -1.85822775e-02  1.82297108e+00 -2.21191404e-08]
 [ 7.31786260e-24  1.60533917e-11 -2.21191401e-08  2.54718760e-01]]

After 15 iterations:
[[ 4.74507887e+00 -1.78120256e-02 -2.11670163e-16  2.38225560e-17]
 [-1.78120256e-02  3.17748431e+00 -1.15075434e-03 -6.87461334e-12]
 [ 1.07356517e-17 -1.15075434e-03  1.82271806e+00  9.36539365e-09]
 [ 6.41677201e-26 -6.87407266e-12  9.36539388e-09  2.54718760e-01]]

After 20 iterations:
[[ 4.74527757e+00 -2.39738535e-03 -2.32066522e-16  2.91364231e-17]
 [-2.39738535e-03  3.17728658e+00 -7.149

## Jacobi

In [3]:
def Gpq(A,p,q): ## (p, q) means (row, column)
    app=A[p-1,p-1]
    apq=A[p-1,q-1]
    aqp=A[q-1,p-1]
    aqq=A[q-1,q-1]
    eta=(aqq-app)/2/apq
    if eta >= 0: t = 1/(eta+sqrt(1+eta*eta))
    else: t = -1/(sqrt(1+eta*eta)-eta)
    c = 1/sqrt(1+t*t)
    s = c*t
    ## print Gpq
    Gpq = identity(A.shape[0])
    for i in range(A.shape[0]):
        if i == p-1:
            Gpq[i][i]=c
            Gpq[i][q-1]=s
        elif i == q-1:
            Gpq[i][i]=c
            Gpq[i][p-1]=-s
            
    return Gpq

def Jacobi_algorithm(A, iteration_times):
    T=A
    for iter in range(iteration_times):
        maxnorm=0
        
        for i in range(T.shape[0]): ## i = row index = row-1 = p-1
            for j in range(i+1, T.shape[1]): ## j = column index = column-1 = q-1
                if abs(T[i,j])>maxnorm:
                    maxnorm = abs(T[i,j])
                    p=i+1
                    q=j+1
                    
        G=Gpq(T,p,q)
        T=(G.T).dot((T.dot(G)))
                
    return T


for i in iteration:
    print("After %d iterations:"%(i))
    print(Jacobi_algorithm(T,i))
    print()


After 5 iterations:
[[ 3.02671876e-01  1.98877843e-01  2.31991360e-17  3.90083050e-01]
 [ 1.98877843e-01  3.15387625e+00 -1.13689848e-01  2.65066327e-18]
 [-5.24114785e-17 -1.13689848e-01  1.84612375e+00  1.98877843e-01]
 [ 3.90083050e-01  1.15838509e-16  1.98877843e-01  4.69732812e+00]]

After 10 iterations:
[[ 2.54777527e-01  8.28956379e-04 -9.51832874e-03 -1.84018719e-03]
 [ 8.28956379e-04  3.17728268e+00 -4.96426137e-06  1.04874699e-18]
 [-9.51832874e-03 -4.96426137e-06  1.82265953e+00  8.17755211e-04]
 [-1.84018719e-03  1.26509551e-16  8.17755211e-04  4.74528026e+00]]

After 15 iterations:
[[ 2.54718760e-01 -2.52080954e-12  3.41591974e-07  1.92963626e-10]
 [-2.52084558e-12  3.17728292e+00 -3.38365294e-21 -3.41591974e-07]
 [ 3.41591974e-07  1.42230687e-17  1.82271708e+00 -2.52080970e-12]
 [ 1.92963712e-10 -3.41591974e-07 -2.52078833e-12  4.74528124e+00]]

After 20 iterations:
[[ 2.54718760e-01 -4.78940032e-28  1.35988232e-22 -1.95566523e-26]
 [-3.60390423e-17  3.17728292e+00 -1.101

/var/folders/cf/rp73zkws3232x_89lllkwqt00000gm/T/ipykernel_48227/2352768081.py:7: RuntimeWarning: overflow encountered in double_scalars
  if eta >= 0: t = 1/(eta+sqrt(1+eta*eta))


## Sturm

In [7]:
def p_n(A, x, n): ## A is a symmetric tridiagonal matrix
    d=[A[i,i] for i in range(A.shape[0])]
    b=[A[i,i+1] for i in range(A.shape[0]-1)]
    if n==0: return 1
    elif n==1: return d[1-1]-x
    else:
        return (d[n-1]-x)*p_n(A,x,n-1) - b[n-1-1]**2*p_n(A,x,n-2)
    
def smu(A,mu):
    n=A.shape[0]
    Smu=[p_n(A, mu, i) for i in range(n+1)]
    sign_changes=0
    for i in range(1, n+1):
        if Smu[i]==0: sign_changes += 1
        elif Smu[i]*Smu[i-1]<0 or Smu[i]/Smu[i-1]<0:
            sign_changes += 1
    
    return sign_changes

def eigenvalue_lower_bound(A):
    n=A.shape[0]
    d=[A[i,i] for i in range(A.shape[0])]
    b=[A[i,i+1] for i in range(A.shape[0]-1)]
    alpha=d[0] - abs(b[0])
    
    for i in range(1, n):
        if i==n-1: temp=d[i]-abs(b[i-1])
        else: temp=d[i]-abs(b[i-1])-abs(b[i])
        if temp < alpha: alpha = temp
        
    return alpha

def eigenvalue_upper_bound(A):
    n=A.shape[0]
    d=[A[i,i] for i in range(A.shape[0])]
    b=[A[i,i+1] for i in range(A.shape[0]-1)]
    beta=d[0] + abs(b[0])
    
    for i in range(1,n):
        if i==n-1: temp=d[i]+abs(b[i-1])
        else: temp=d[i]+abs(b[i-1])+abs(b[i])
        if temp > beta: beta = temp
        
    return beta

def Sturm_algorithm(A, iteration_times): ## No.i max eigenvalue
    n=A.shape[0]
    eigenvalue=[0 for i in range(n)]
    for i in range(1,n+1):
        a=eigenvalue_lower_bound(A)
        b=eigenvalue_upper_bound(A)
        iter=0
        while True:
            c=(a+b)/2
            if smu(A,c)>n-i: b=c
            else: a=c
            iter += 1
            if iter==iteration_times: break
        eigenvalue[i-1]=c

    return eigenvalue

print("eigenvalues_lower_bound:",eigenvalue_lower_bound(T))
print("eigenvalues_upper_bound:",eigenvalue_upper_bound(T))
print()
for i in iteration:
    print("After %d iterations:"%(i))
    print("Approximate eigenvalues:",Sturm_algorithm(T, i))
    print()

eigenvalues_lower_bound: 0
eigenvalues_upper_bound: 5

After 5 iterations:
Approximate eigenvalues: [4.84375, 3.28125, 1.71875, 0.15625]

After 10 iterations:
Approximate eigenvalues: [4.7412109375, 3.1787109375, 1.8212890625, 0.2587890625]

After 15 iterations:
Approximate eigenvalues: [4.745330810546875, 3.177337646484375, 1.822662353515625, 0.254669189453125]

After 20 iterations:
Approximate eigenvalues: [4.745278358459473, 3.1772851943969727, 1.8227148056030273, 0.25472164154052734]

After 50 iterations:
Approximate eigenvalues: [4.7452812401741395, 3.1772829191128915, 1.8227170808871085, 0.25471875982586045]

After 100 iterations:
Approximate eigenvalues: [4.745281240174139, 3.1772829191128915, 1.822717080887108, 0.254718759825861]



# 2. 幂次法求矩阵最大模的本征值和本征

In [ ]:
from numpy import dot
from numpy import identity
from numpy import zeros
from numpy import shape
from numpy import sqrt
from numpy import array

def delta(i,j):
    if i==j: return 1
    else: return 0

N=10

A=[[-delta(i-1,j)-delta(i+1,j)+2*delta(i,j) for j in range(0,N)] for i in range(0,N)]
A=array(A)

## (b)

In [ ]:
epsilon=1e-16

def power_method(A):
    q=[zeros(shape=N)]
    q[0][0]=1
    z=[zeros(shape=N)]
    nu=[0]
    i=1
    while True:
        z.append(array(A.dot(q[i-1])))
        q.append(array(z[i]/sqrt(z[i].T.dot(z[i]))))
        nu.append((q[i].T).dot(A).dot(q[i]))
        if abs(nu[i]-nu[i-1])<epsilon: break
        i += 1
    return nu[i], q[i]

In [ ]:
eigenvalue1,eigenvector1=power_method(A)
print("最大本征值为:",eigenvalue1)
print("对应本征矢量为:",eigenvector1)

# 3.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def Lorentz_attractor(beta=8/3,rho=28,sigma=10):
    nstep=100000
    x_final=10
    x_initial=0
    x_step=(x_final-x_initial)/nstep
    x=[12]
    y=[4]
    z=[0]
    for i in range(nstep):
        x.append(x[i] + x_step*(-beta*x[i]+y[i]*z[i]))
        y.append(y[i] + x_step*(-sigma*y[i]+sigma*z[i]))
        z.append(z[i] + x_step*(-y[i]*x[i]+rho*y[i]-z[i]))
    
    return x,y,z

In [ ]:
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel(r'$y_1$',fontsize=16)
ax.set_ylabel(r'$y_2$',fontsize=16)
ax.set_zlabel(r'$y_3$',fontsize=16)
x,y,z=Lorentz_attractor()
ax.plot3D(x, y, z, 'darkorange')
plt.savefig("./Problem3(a).pdf")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel(r'$y_1$',fontsize=16)
ax.set_ylabel(r'$y_2$',fontsize=16)
ax.set_zlabel(r'$y_3$',fontsize=16)
x,y,z=Lorentz_attractor(5/3,28,10)
ax.plot3D(x, y, z, 'darkorange')
plt.savefig("./Problem3(b).pdf")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel(r'$y_1$',fontsize=16)
ax.set_ylabel(r'$y_2$',fontsize=16)
ax.set_zlabel(r'$y_3$',fontsize=16)
x,y,z=Lorentz_attractor(rho=8)
ax.plot3D(x, y, z, 'darkorange')
plt.savefig("./Problem3(c).pdf")
plt.show()